In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model

In [3]:
df = pd.read_csv("hyundai.csv")
df

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,I20,2017,7999,Manual,17307,Petrol,145,58.9,1.2
1,Tucson,2016,14499,Automatic,25233,Diesel,235,43.5,2.0
2,Tucson,2016,11399,Manual,37877,Diesel,30,61.7,1.7
3,I10,2016,6499,Manual,23789,Petrol,20,60.1,1.0
4,IX35,2015,10199,Manual,33177,Diesel,160,51.4,2.0
...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,0,78.4,1.6
4856,I40,2015,7830,Manual,59508,Diesel,30,65.7,1.7
4857,I10,2017,6830,Manual,13810,Petrol,20,60.1,1.0
4858,Tucson,2018,13994,Manual,23313,Petrol,145,44.8,1.6


In [4]:
inputs = df.drop(['tax(£)', 'engineSize'], axis='columns')
target = df.price
target

0        7999
1       14499
2       11399
3        6499
4       10199
        ...  
4855     8680
4856     7830
4857     6830
4858    13994
4859    15999
Name: price, Length: 4860, dtype: int64

In [5]:
from sklearn.preprocessing import LabelEncoder
Numerics = LabelEncoder()

In [7]:
inputs['model_n']=Numerics.fit_transform(inputs['model'])
inputs['transmission_n']=Numerics.fit_transform(inputs['transmission'])
inputs['fueltype_n']=Numerics.fit_transform(inputs['fuelType'])
inputs


,model,year,price,transmission,mileage,fuelType,mpg,model_n,transmission_n,fueltype_n
0,I20,2017,7999,Manual,17307,Petrol,58.9,4,1,3
1,Tucson,2016,14499,Automatic,25233,Diesel,43.5,14,0,0
2,Tucson,2016,11399,Manual,37877,Diesel,61.7,14,1,0
3,I10,2016,6499,Manual,23789,Petrol,60.1,3,1,3
4,IX35,2015,10199,Manual,33177,Diesel,51.4,9,1,0
...,...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,78.4,5,1,0
4856,I40,2015,7830,Manual,59508,Diesel,65.7,6,1,0
4857,I10,2017,6830,Manual,13810,Petrol,60.1,3,1,3
4858,Tucson,2018,13994,Manual,23313,Petrol,44.8,14,1,3


In [9]:
inputs_n = inputs.drop(['model', 'transmission', 'fuelType'], axis='columns')
inputs_n

,year,price,mileage,mpg,model_n,transmission_n,fueltype_n
0,2017,7999,17307,58.9,4,1,3
1,2016,14499,25233,43.5,14,0,0
2,2016,11399,37877,61.7,14,1,0
3,2016,6499,23789,60.1,3,1,3
4,2015,10199,33177,51.4,9,1,0
...,...,...,...,...,...,...,...
4855,2016,8680,25906,78.4,5,1,0
4856,2015,7830,59508,65.7,6,1,0
4857,2017,6830,13810,60.1,3,1,3
4858,2018,13994,23313,44.8,14,1,3


In [10]:
model = linear_model.LinearRegression()

In [12]:
model.fit(inputs_n, target)

LinearRegression()

In [13]:
pred=model.predict([[2017, 7999, 17307, 58.9, 5, 1, 3]])
pred

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=[2.0170e+03 7.9990e+03 1.7307e+04 5.8900e+01 5.0000e+00 1.0000e+00
 3.0000e+00].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.